# Install & Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install bitsandbytes==0.41.1
!pip install accelerate
!pip install appdirs
!pip install loralib
!pip install black black[jupyter]
!pip install datasets
!pip install fire
!pip install git+https://github.com/huggingface/peft
!pip install transformers==4.34.1
!pip install sentencepiece sentence_transformers
!pip install scipy numpy scikit-learn pandas
!pip install datasets==2.17.1
!pip install -U huggingface_hub
!pip install pyarrow==15.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=e3bf3f2d04e80389f5bc9f50addd8fd890bcc1b64619d107390aa177cc712283
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-_9x1cwhj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-_9x1cw

In [3]:
import os
import os.path as osp
import sys
import fire
import json
from typing import List, Union

import torch
from torch.nn import functional as F

import transformers
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

from datasets import load_dataset
import pandas as pd

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_int8_training,
    set_peft_model_state_dict
)
from peft import PeftModel

from pyarrow import json
import pyarrow.parquet as pq
import pyarrow as pa

In [4]:
# pip install시 utf-8, ansi 관련 오류날 경우 필요한 코드
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [5]:
!pip -q install langchain chromadb==0.4.14 openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.17.1 requires huggingface-hub>=0.19.4, but you have huggingface-hub 0.17.3 which is incompatible.


In [6]:
!pip install jq

# LLM 로드 & 미세조정

### 모델, 토크나이저, 훈련셋 로드

In [7]:
base_LLM_model = "upstage/SOLAR-10.7B-v1.0"
device = "auto"

In [ ]:
!pip install accelerate

In [14]:
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/


In [15]:
model = AutoModelForCausalLM.from_pretrained(
    base_LLM_model,
    load_in_8bit=True, # LoRA
    #load_in_4bit=True, # Quantization Load
    torch_dtype=torch.float16,
    device_map=device)

tokenizer = AutoTokenizer.from_pretrained(base_LLM_model)

ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [ ]:
# Check special token
bos = tokenizer.bos_token_id # 문장 시작 토큰
eos = tokenizer.eos_token_id # 문장 끝 토큰
pad = tokenizer.pad_token_id # 문장 패딩 토큰
tokenizer.padding_side = "right" # 패딩 오른쪽

print("BOS token:", bos) # 1
print("EOS token:", eos) # 2
print("PAD token:", pad) # None

In [ ]:
if (pad == None) or (pad == eos):
    tokenizer.pad_token_id = 0  # 만약 패딩값이 없거나 eos값과 같다면,
print("length of tokenizer:",len(tokenizer)) # 32000

In [ ]:
from pyarrow import json
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

# input csv file path
csv_file = "./data/train.csv"

# output parquet file path
parquet_file = "./data/train.parquet"
chunk_size = 100000
header = ['instruction','input','output']
dtype = {
    'instruction':'str',
    'input':'str',
    'output':'str'
}

# parquet 스키마
parquet_schema = pa.schema([
    ('instruction', pa.string()),
    ('input', pa.string()),
    ('output',pa.string())
])

In [ ]:
csv_stream = pd.read_csv(csv_file, chunksize=chunk_size, names=header, dtype=dtype, low_memory=False)

In [14]:
# csv_stream을 반복문으로 가져오며, 각 chunk에 대한 작업
for i, chunk in enumerate(csv_stream):
    print(f"chunk : {i}")

    # 첫 번째 조각인 경우 parquet_writer 객체 생성
    if i == 0:
        parquet_writer = pq.ParquetWriter(parquet_file, parquet_schema, compression='snappy')

    table = pa.Table.from_pandas(chunk, schema=parquet_schema)
    parquet_writer.write_table(table)

parquet_writer.close()

chunk : 0


### Load Dataset

In [ ]:
data = load_dataset("parquet", data_files={'train': './data/train.parquet'})
print(data['train'])

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 6441
})


In [ ]:
instruct_template = {
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. \
                      Write a response that appropriately completes the request.\
                      \n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",

    "prompt_no_input": "Below is an instruction that describes a task. \
                        Write a response that appropriately completes the request.\
                        \n\n### Instruction:\n{instruction}\n\n### Response:\n",

    "response_split": "### Response:"
}

In [ ]:
# 데이터셋과 훈련 횟수와 관련된 하이퍼 파라미터
batch_size = 16
num_epochs = 1
micro_batch = 1
gradient_accumulation_steps = batch_size // micro_batch

# 훈련 방법에 대한 하이퍼 파라미터
cutoff_len = 4096
lr_scheduler = 'cosine'
warmup_ratio = 0.06 # warmup_steps = 100
learning_rate = 4e-4
optimizer = 'adamw_torch'
weight_decay = 0.01
max_grad_norm = 1.0

# LoRA config
lora_r = 16
lora_alpha = 16
lora_dropout = 0.05
lora_target_modules = ["gate_proj", "down_proj", "up_proj"]

# Tokenizer에서 나오는 input값 설정 옵션
train_on_inputs = False
add_eos_token = False

# Others
resume_from_checkpoint = False # !! 만약 모델을 이어서 훈련하고 싶다면, './custom_LLM/checkpoint-[xxx]'와 같이 파일 경로를 입력해야 합니다!

### 관련 함수/하이퍼파라미터 설정/Finetuning
- output_dir = (tokenizer save 경로)
- model_save_path =  (checkpoint 경로)

In [ ]:
output_dir = "./config/tokenizer"#"/content/drive/MyDrive/AIKU_HansolDeco/LLM_finetuning"
model_save_path = "./config/checkpoints"#"/content/drive/MyDrive/AIKU_HansolDeco/data/LLM"

In [ ]:
class Prompter(object):

    def __init__(self, verbose: bool = False):
        self.template = instruct_template

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:

        if input: # input text가 있다면
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )

        if label:
            res = f"{res}{label}"

        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

prompter = Prompter()

In [ ]:
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )

    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):

        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"])

    tokenized_full_prompt = tokenize(full_prompt)
    if not train_on_inputs:

        user_prompt = prompter.generate_prompt(
            data_point["instruction"], data_point["input"])

        tokenized_user_prompt = tokenize(
            user_prompt, add_eos_token=add_eos_token)

        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        tokenized_full_prompt["labels"] = [
            -100
        ] * user_prompt_len + tokenized_full_prompt["labels"][
            user_prompt_len:
        ]
    return tokenized_full_prompt

In [ ]:
val_data = None
train_data = data["train"].shuffle() # random
train_data = train_data.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/6441 [00:00<?, ? examples/s]

In [ ]:
config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM")

In [19]:
model = prepare_model_for_int8_training(model)
model = get_peft_model(model, config)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:145: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


NameError: name 'config' is not defined

In [ ]:
if resume_from_checkpoint:
    checkpoint_name = os.path.join(
        resume_from_checkpoint, "pytorch_model.bin"
    )  # All checkpoint

    if not os.path.exists(checkpoint_name):
        checkpoint_name = os.path.join(
            resume_from_checkpoint, "adapter_model.bin"
        )  # only LoRA model
        resume_from_checkpoint = (
            True
        ) # kyujin: I will use this checkpoint

    if os.path.exists(checkpoint_name):
        print(f"Restarting from {checkpoint_name}")
        adapters_weights = torch.load(checkpoint_name)
        set_peft_model_state_dict(model, adapters_weights)

    else:
        print(f"Checkpoint {checkpoint_name} not found")

In [ ]:
trainer = transformers.Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args=transformers.TrainingArguments( # 훈련에 이용될 하이퍼파라미터
            per_device_train_batch_size = micro_batch,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_ratio=warmup_ratio,
            num_train_epochs=num_epochs,
            learning_rate=learning_rate,
            fp16=True,
            logging_steps=1,
            optim="adamw_torch",
            evaluation_strategy="no",
            save_strategy="steps",
            max_grad_norm = max_grad_norm,
            save_steps = 30, # you can change!
            lr_scheduler_type=lr_scheduler,
            output_dir=output_dir,
            save_total_limit=2,
            load_best_model_at_end=False,
            ddp_find_unused_parameters=False,
            group_by_length = False
        ),
        data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
    )

model.config.use_cache = False
model.print_trainable_parameters() # 훈련하는 파라미터의 % 체크

if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

NameError: name 'train_data' is not defined

In [ ]:
torch.cuda.empty_cache()
trainer.train(resume_from_checkpoint=resume_from_checkpoint)

In [ ]:
model.save_pretrained(model_save_path)
model_path = os.path.join(output_dir, "pytorch_model.bin")
torch.save({}, model_path)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/AIKU_HansolDeco/LLM_finetuning/tokenizer_config.json',
 '/content/drive/MyDrive/AIKU_HansolDeco/LLM_finetuning/special_tokens_map.json',
 '/content/drive/MyDrive/AIKU_HansolDeco/LLM_finetuning/tokenizer.model',
 '/content/drive/MyDrive/AIKU_HansolDeco/LLM_finetuning/added_tokens.json',
 '/content/drive/MyDrive/AIKU_HansolDeco/LLM_finetuning/tokenizer.json')

### Adapter Merge

In [29]:
torch.cuda.empty_cache()

base_model = AutoModelForCausalLM.from_pretrained(
    base_LLM_model,
    return_dict = True,
    torch_dtype=torch.float16,
    device_map=device)

model = PeftModel.from_pretrained(base_model, model_save_path, device)
model = model.merge_and_unload()

print(model)

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

NameError: name 'model_save_path' is not defined

# RAG 결합

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma
from langchain.schema.runnable import RunnablePassthrough
from transformers import pipeline

### Retrieval 대상 문서 만들기 (완료. 코드 실행 불필요)

In [ ]:
# TEST_SET_PATH = "/content/drive/MyDrive/AIKU_HansolDeco/data/test.csv"
# TRAIN_SET_PATH = "/content/drive/MyDrive/AIKU_HansolDeco/data/train.csv"
# GENERATED_DOC_PATH = "/content/drive/MyDrive/AIKU_HansolDeco/"

In [ ]:
# import pandas as pd

# df1 = pd.read_csv(TEST_SET_PATH)
# df2 = pd.read_csv(TRAIN_SET_PATH)

# questions = list()
# for i in df1["질문"]:
#   questions.append(i)
# for j in df2["질문_2"]:
#   questions.append(j)

# len(questions)

In [ ]:
# for i in questions[:5]:
#   print(i)

In [ ]:
# from openai import OpenAI
# API_KEY = "GPT KEY" # 발급 받은 API KEY 입력
# client = OpenAI(api_key=API_KEY)

In [ ]:
# import time

# resp = list()

# for question in questions:

#   completion = client.chat.completions.create(
#   model="gpt-4-turbo-preview",
#   messages=[
#             {"role": "system", "content": "너는 도배, 건축물, 벽지 공사, 인테리어 영역에 대한 실무부터 이론까지 모두 파악하고 있는 전문가야. 앞선 영역과 관련된 질문에 대해 할 수 있는 한 최대한 자세하고 전문적인 에세이를 작성해. '~습니다', '~입니다' 말투 대신 '~이다.' 말투로 에세이를 작성하도록 해. 한국어로 작성해."},
#             {"role": "user", "content": f"{question}"}
#             ],
#   max_tokens=4096,
#   temperature=0.2
#   )

#   resp.append(completion.choices[0].message)
#   time.sleep(3)

# len(resp)

In [ ]:
# len(resp)

In [ ]:
# docs = list()

# for i in resp:
#   docs.append(i.content)

# docs[0]

In [ ]:
# import json

# with open("/content/drive/MyDrive/generated_doc.json", 'w') as fp:
#   json.dump(docs, fp, ensure_ascii=False)

In [ ]:
# len(docs)

In [ ]:
# for i in resp:
#   print(i.content)
#   break

### Vector DB 생성 및 로드

In [ ]:
loader = JSONLoader(
    file_path="/content/drive/MyDrive/AIKU_HansolDeco/chroma/generated_doc.json",
    jq_schema='.[]',
    )

data = loader.load()
print(data)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(docs)

In [ ]:
model_name = "jhgan/ko-sroberta-multitask"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

vectordb = Chroma.from_documents(documents=texts, embedding=ko_embedding, persist_directory="/content/drive/MyDrive/AIKU_HansolDeco/chroma")

In [ ]:
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={'k': 3})

In [ ]:
r = vectordb.similarity_search("페인트의 종류에 대해 자세히 설명해줘", k = 3)
print(len(r))
for i in r:
  print(i.page_content)
  print('-'*50)

### Chain 생성

In [ ]:
# model -> 미세조정된 SOLAR 불러오기
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=10000,
)

prompt_template = """
### [INST]
Instruction: Answer the question based on your knowledge.
Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
 """

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [ ]:
a = rag_chain.invoke("페인트의 종류에 대해서 자세히 설명해줘")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [ ]:
a['text'].strip('\n#ANSWER :')

'페인트의 종류로는 수성페인트, 유성페인트, 천연페인트 등이 있습니다. 이 중에서도 각각의 페인트는 특정한 용도와 장단점을 가지고 있습니다. 수성페인트는 물을 기반으로 제조된 페인트로, 냄새가 적고 빠르게 건조되는 장점이 있습니다. 유성페인트는 유기 용제를 사용하여 제조된 페인트로, 냄새가 강하고 건조 시간이 길다는 단점이 있습니다. 마지막으로 천연페인트는 천연 원료를 사용하여 제조된 페인트로, 환경에 친화적이며 냄새가 적은 장점이 있습니다. 이러한 페인트의 종류는 각각의 특성에 따라 사용되며, 목재, 철재, �'

# Inference 및 제출파일 내보내기 - only SOLAR

### Inference

In [ ]:
import pandas as pd
test_data = pd.read_csv("/content/drive/MyDrive/AIKU_HansolDeco/data/test.csv")

In [ ]:
from tqdm import tqdm
question_lst = test_data['질문'].tolist()
conjunction_list = ["또한", "그리고"]


original_output_txt_lst = [] # 후처리 전 원본 텍스트 저장
output_txt_lst = [] # 후처리 후 텍스트 저장
for question in tqdm(question_lst):
    for c in conjunction_list:
        question_lst = question.split(c)
        if len(question_lst) > 1:break

    assert len(question_lst) > 0
    answer_lst = []
    for idx,q_ in enumerate(question_lst):

        ## 접속사 처리
        #if idx != 0 :
        #    for c in connect_list :
        #        q_ = q_[1:].replace(c,'')
        #q_ += "?"
        q_ = q_.strip(', ')

        ## 각 질문 별로 답변 생성
        #inputs = tokenizer(q_, return_tensors="pt").to(model.device)
        #outputs = model.generate(**inputs, max_new_tokens=500)
        #full_text = tokenizer.decode(outputs[0])
        full_text = rag_chain.invoke(f"{q_}")

        # 문장 끊길 때
        if full_text[3:-4][-1] != '.' : answer_lst.append(full_text[3:-4].replace(q_,'').strip('\n?, ') + "등이 있습니다.")
        else : answer_lst.append(full_text[3:-4].replace(q_,'').strip())

    answer_sentence = ' 그리고 '.join(answer_lst)

    output_txt_lst.append(answer_sentence)
    print(question)
    print(output_txt_lst[-1])
    original_output_txt_lst.append(full_text)

  0%|          | 0/130 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 0/130 [00:04<?, ?it/s]


KeyboardInterrupt: 

### 제출파일 내보내기

In [ ]:
index_lst = [ "TEST_" + "{:03d}".format(i) for i in range(0,130)]
df = pd.DataFrame({"id" : index_lst,"Answer" : output_txt_lst})
df = df.set_index("id")

In [ ]:
df

In [ ]:
# 임베딩 변환 전 결과 csv로 저장 (한국어 텍스트)
df.to_csv("./result/rag_answer.csv",encoding="utf-8-sig")

In [ ]:
from sentence_transformers import SentenceTransformer

inf_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# embedding 변환
pred_embeddings = inf_model.encode(output_txt_lst)
assert pred_embeddings.shape[0] == 130

# 결과물 저장 -> 겹치지 않게 csv 명 파일마다 바꿔주기
submit = pd.read_csv('./data/sample_submission.csv')
submit.iloc[:,1:] = pred_embeddings
submit.head()

submit.to_csv('./result/rag_submission.csv', index=False)

# Inference 및 제출 파일 내보내기 - SOLAR + RAG

In [23]:
import pandas as pd
test_data = pd.read_csv("./data/test.csv")

In [25]:
save_filename = './result/result.txt'
w = open(save_filename, 'a')

In [26]:
question_lst = test_data['질문'].tolist()

In [ ]:
question_lst[3]

'철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?'

In [45]:
a = (rag_chain.invoke(question_lst[33]))['text']

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.63 GiB. GPU 0 has a total capacty of 39.56 GiB of which 2.63 GiB is free. Process 10555 has 36.93 GiB memory in use. Of the allocated memory 30.76 GiB is allocated by PyTorch, and 5.66 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [42]:
a

'\n### ANSWER:\n프리케스트 콘크리트 구조는 건축물의 벽, 바닥, 기둥, 보 등의 부재를 공장에서 미리 제작하여 현장에서 조립하는 방식을 말합니다. 이 방식은 건축물의 내구성과 내화성을 높이는 데 유용하며, 주차장, 아파트, 상업 건물 등에 적합합니다. 반면, 조적식 구조는 벽돌, 석재, 콘크리트 블록과 같은 조적 단위를 사용하여 벽을 쌓아 올리는 전통적인 건축 방식으로, 유연성이 뛰어나며 건축물의 미적 가치를 높일 수 있습니다. 조적식 구조는 비구조적 벽이나 저층 건물의 구조적 벽에 주로 사용됩니다.'

In [ ]:
from tqdm import tqdm
question_lst = test_data['질문'].tolist()
conjunction_list = ["또한", "그리고"]


output_txt_lst = [] # 후처리 후 텍스트 저장

for question in tqdm(question_lst):
    #breakpoint()
    tmp_answer = rag_chain.invoke(question)
    output_txt = tmp_answer['text'].strip('\n#ANSWER :')
    if type(output_txt) != 'str' : output_txt = str(output_txt)

    w.write(output_txt + '\n')
    output_txt_lst.append(output_txt)
    print(output_txt_lst[-1])

w.close()

  0%|          | 0/130 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  1%|          | 1/130 [04:23<9:27:26, 263.93s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알루미늄페인트, 역청질페인트, 워시프라이머, 규산염페인트, 규산염 알루미늄페인트, 아연 인산페인트, 아연 알루미늄페인트, 아연 아연페인트, 아연 아연 규산염페인트, 아연 아연 알루미늄페인트, 아연 아연 아연페인트, 아연 아연 아연 규산염페인트, 아연 아연 아연 알루미늄페인트, 아연 아연 아연 아연페인트, 아연 아연 아연 아연 규산염페인트, 아연 아연 아연 아연 알루미늄페인트, 아연 아연 아연 아연 아연페인트, 아연


  2%|▏         | 2/130 [08:57<9:35:32, 269.79s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


도배지에 녹은 자국이 발생하는 원인은 습기와 곰팡이, 금속성 부속물의 부식 등이 있습니다. 습기로 인해 도배지 뒷면에 곰팡이가 생기거나 금속 부속물이 부식되어 도배지에 녹은 자국이 발생할 수 있습니다. 이러한 문제를 해결하기 위해서는 먼저 원인을 파악하고 적절한 방법으로 처리해야 합니다. 습기 관리를 위해 제습기를 가동하고 환기를 통해 습기를 제거하는 것이 중요하며, 곰팡이가 발견되면 곰팡이 제거제를 사용하여 곰팡이를 제거하고 재발방지를 위해 방습도료를 도포하


  2%|▏         | 3/130 [10:40<6:49:37, 193.53s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

수성 벽체용 탄성 방수 도료는 평지붕의 누수 문제를 방지하는 데 매우 효과적인 솔루션이다. 이 도료는 환경 친화적이며, 우수한 탄성과 복원력을 가지고 있어 누수를 방지하는 데 도움을 준다. 또한 적용이 용이하고 미적 다양성이 뛰어나며, 유지 보수가 용이하다는 점도 장점으로 꼽힙니다.


 99%|█████████▉| 129/130 [6:27:49<03:03, 183.01s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


석고수정이 발생하는 가장 큰 원인은 실내 습도가 높은 경우입니다. 습도가 높으면 석고보드에 포함된 석고가 수분을 흡수하여 결정화되어 표면에 흰색 가루 형태로 나타나게 됩니다. 따라서 습도 관리가 중요하며, 환기를 통해 실내 습도를 적정 수준으로 유지하는 것이 중요합니다. 또한, 건축 자재의 품질과 시공 방법도 석고수정 현상을 유발할 수 있으므로 품질이 높은 건축 자재를 사용하고 적절한 시공 방법을 준수하는 것이 필요합니다.


100%|██████████| 130/130 [6:28:39<00:00, 179.38s/it]

카페트의 기대수명은 10년입니다. 오리지널징크의 장점은 부식방지, 내구성, 경제성, 환경친화성, 다양한 용도에서 찾을 수 있습니다.


In [4]:
with open('./result/result_rag.txt','r') as f:
    output_txt_lst = f.readlines()

In [5]:
output_txt_lst = list(map(lambda x : (''.join(x.split('.')[1:])).strip('\n '),output_txt_lst))

In [6]:
output_txt_lst = output_txt_lst[1:]

In [7]:
import pandas as pd

In [8]:
index_lst = [ "TEST_" + "{:03d}".format(i) for i in range(0,130)]
df = pd.DataFrame({"id" : index_lst,"Answer" : output_txt_lst})
df = df.set_index("id")

In [9]:
# 임베딩 변환 전 결과 csv로 저장 (한국어 텍스트)
df.to_csv("./result/rag_answer_3.csv",encoding="utf-8-sig")

In [10]:
from sentence_transformers import SentenceTransformer

inf_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# embedding 변환
pred_embeddings = inf_model.encode(output_txt_lst)
assert pred_embeddings.shape[0] == 130

# 결과물 저장 -> 겹치지 않게 csv 명 파일마다 바꿔주기
submit = pd.read_csv('./data/sample_submission.csv')
submit.iloc[:,1:] = pred_embeddings
submit.head()

submit.to_csv('./result/rag_submission_1.csv', index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]